In [9]:
from ngraph.scenario import Scenario
from ngraph.traffic_demand import TrafficDemand
from ngraph.traffic_manager import TrafficManager
from ngraph.lib.flow_policy import FlowPolicyConfig, FlowPolicy, FlowPlacement
from ngraph.lib.algorithms.base import PathAlg, EdgeSelect
from ngraph.failure_manager import FailureManager
from ngraph.failure_policy import FailurePolicy, FailureRule, FailureCondition
from ngraph.explorer import NetworkExplorer

In [10]:
scenario_yaml = """
blueprints:
  server_pod:
    rsw:
        node_count: 48
        attrs:
          hw_component: Minipack2_128x200GE
  
  f16_2tier:
    groups:
      ssw:
        node_count: 36
        attrs:
          hw_component: Minipack2_128x200GE
      fsw:
        node_count: 96
        attrs:
          hw_component: Minipack2_128x200GE

    adjacency:
      - source: /ssw
        target: /fsw
        pattern: mesh
        link_params:
          capacity: 200
          cost: 1
          
  hgrid_2tier:
    groups:
      fauu:
        node_count: 8
        attrs:
          hw_component: Minipack2_128x200GE
      fadu:
        node_count: 36
        attrs:
          hw_component: Minipack2_128x200GE

    adjacency:
      - source: /fauu
        target: /fadu
        pattern: mesh
        link_params:
          capacity: 400
          cost: 1

  fa:
    groups:
      fa1:
        use_blueprint: hgrid_2tier
      fa2:
        use_blueprint: hgrid_2tier
      fa3:
        use_blueprint: hgrid_2tier
      fa4:
        use_blueprint: hgrid_2tier
      fa5:
        use_blueprint: hgrid_2tier
      fa6:
        use_blueprint: hgrid_2tier
      fa7:
        use_blueprint: hgrid_2tier
      fa8:
        use_blueprint: hgrid_2tier
      fa9:
        use_blueprint: hgrid_2tier
      fa10:
        use_blueprint: hgrid_2tier
      fa11:
        use_blueprint: hgrid_2tier
      fa12:
        use_blueprint: hgrid_2tier
      fa13:
        use_blueprint: hgrid_2tier
      fa14:
        use_blueprint: hgrid_2tier
      fa15:
        use_blueprint: hgrid_2tier
      fa16:
        use_blueprint: hgrid_2tier
                  
  dc_fabric:
    groups:
      plane1:
        use_blueprint: f16_2tier
      plane2:
        use_blueprint: f16_2tier
      plane3:
        use_blueprint: f16_2tier
      plane4:
        use_blueprint: f16_2tier
      plane5:
        use_blueprint: f16_2tier
      plane6:
        use_blueprint: f16_2tier
      plane7:
        use_blueprint: f16_2tier
      plane8:
        use_blueprint: f16_2tier

      pod1:
        use_blueprint: server_pod
      pod36:
        use_blueprint: server_pod
    
    adjacency:
        - source: /pod1/rsw
          target: /plane[0-9]*/fsw/fsw-1
          pattern: mesh
          link_params:
            capacity: 200
            cost: 1
        - source: /pod36/rsw
          target: /plane[0-9]*/fsw/fsw-36
          pattern: mesh
          link_params:
            capacity: 200
            cost: 1

  ebb:
    groups:
      eb01:
        node_count: 4
      eb02:
        node_count: 4
      eb03:
        node_count: 4
      eb04:
        node_count: 4
      eb05:
        node_count: 4
      eb06:
        node_count: 4
      eb07:
        node_count: 4
      eb08:
        node_count: 4  

    adjacency:
      - source: eb01
        target: eb01
        pattern: mesh
        link_params: { capacity: 3200, cost: 10 }
      - source: eb02
        target: eb02
        pattern: mesh
        link_params: { capacity: 3200, cost: 10 }
      - source: eb03
        target: eb03
        pattern: mesh
        link_params: { capacity: 3200, cost: 10 }
      - source: eb04
        target: eb04
        pattern: mesh
        link_params: { capacity: 3200, cost: 10 }
      - source: eb05
        target: eb05
        pattern: mesh
        link_params: { capacity: 3200, cost: 10 }
      - source: eb06
        target: eb06
        pattern: mesh
        link_params: { capacity: 3200, cost: 10 }
      - source: eb07
        target: eb07
        pattern: mesh
        link_params: { capacity: 3200, cost: 10 }
      - source: eb08
        target: eb08
        pattern: mesh
        link_params: { capacity: 3200, cost: 10 }
             
network:
  name: "fb_region"
  version: 1.0

  groups:
    dc1:
      use_blueprint: dc_fabric
    dc2:
      use_blueprint: dc_fabric
    dc3:
      use_blueprint: dc_fabric
    dc5:
      use_blueprint: dc_fabric
    dc6:
      use_blueprint: dc_fabric

    fa:
        use_blueprint: fa

    ebb:
        use_blueprint: ebb

  adjacency:
        - source: .*/ssw/.*
          target: .*/fa1/fadu/.*
          pattern: one_to_one
          link_params:
            capacity: 200
            cost: 1 
        - source: .*/ssw/.*
          target: .*/fa2/fadu/.*
          pattern: one_to_one
          link_params:
            capacity: 200
            cost: 1 
        - source: .*/ssw/.*
          target: .*/fa3/fadu/.*
          pattern: one_to_one
          link_params:
            capacity: 200
            cost: 1
        - source: .*/ssw/.*
          target: .*/fa4/fadu/.*
          pattern: one_to_one
          link_params:
            capacity: 200
            cost: 1    
        - source: .*/ssw/.*
          target: .*/fa5/fadu/.*
          pattern: one_to_one
          link_params:
            capacity: 200
            cost: 1    
        - source: .*/ssw/.*
          target: .*/fa6/fadu/.*
          pattern: one_to_one
          link_params:
            capacity: 200
            cost: 1    
        - source: .*/ssw/.*
          target: .*/fa7/fadu/.*
          pattern: one_to_one
          link_params:
            capacity: 200
            cost: 1    
        - source: .*/ssw/.*
          target: .*/fa8/fadu/.*
          pattern: one_to_one
          link_params:
            capacity: 200
            cost: 1
        - source: .*/ssw/.*
          target: .*/fa9/fadu/.*
          pattern: one_to_one
          link_params:
            capacity: 200
            cost: 1 
        - source: .*/ssw/.*
          target: .*/fa10/fadu/.*
          pattern: one_to_one
          link_params:
            capacity: 200
            cost: 1 
        - source: .*/ssw/.*
          target: .*/fa11/fadu/.*
          pattern: one_to_one
          link_params:
            capacity: 200
            cost: 1
        - source: .*/ssw/.*
          target: .*/fa12/fadu/.*
          pattern: one_to_one
          link_params:
            capacity: 200
            cost: 1    
        - source: .*/ssw/.*
          target: .*/fa13/fadu/.*
          pattern: one_to_one
          link_params:
            capacity: 200
            cost: 1    
        - source: .*/ssw/.*
          target: .*/fa14/fadu/.*
          pattern: one_to_one
          link_params:
            capacity: 200
            cost: 1    
        - source: .*/ssw/.*
          target: .*/fa15/fadu/.*
          pattern: one_to_one
          link_params:
            capacity: 200
            cost: 1    
        - source: .*/ssw/.*
          target: .*/fa16/fadu/.*
          pattern: one_to_one
          link_params:
            capacity: 200
            cost: 1
        - source: .*/fauu-[15]
          target: .*/eb0[1-8]-1
          pattern: mesh
          link_count: 2
          link_params:
            capacity: 400
            cost: 1
        - source: .*/fauu-[26]
          target: .*/eb0[1-8]-2
          pattern: mesh
          link_count: 2
          link_params:
            capacity: 400
            cost: 1   
        - source: .*/fauu-[37]
          target: .*/eb0[1-8]-3
          pattern: mesh
          link_count: 2
          link_params:
            capacity: 400
            cost: 1 
        - source: .*/fauu-[48]
          target: .*/eb0[1-8]-4
          pattern: mesh
          link_count: 2
          link_params:
            capacity: 400
            cost: 1  
components:
  Minipack2_128x200GE:
    component_type: chassis
    power_watts: 1750  # typical power consumption with 128x200GE QSFP56 200G-FR4 at 30C
  QSFP56_200G-FR4_2km:
    component_type: pluggable_optics
    power_watts: 6.5  
"""
scenario = Scenario.from_yaml(scenario_yaml)
network = scenario.network

In [11]:
network.nodes["dc1/plane1/ssw/ssw-1"]

Node(name='dc1/plane1/ssw/ssw-1', attrs={'hw_component': 'Minipack2_128x200GE', 'type': 'node', 'disabled': False})

In [12]:
comp_lib = scenario.components_library
comp_lib.get("Minipack2_128x200GE")

Component(name='Minipack2_128x200GE', component_type='chassis', cost=0.0, power_watts=1750.0, ports=0, children={})

In [13]:
network.max_flow(
    source_path=".*/fsw.*",
    sink_path=".*/eb.*",
    mode="combine",
    shortest_path=True,
)

{('.*/fsw.*', '.*/eb.*'): 819200.0}

In [14]:
# Profiling
import cProfile
import pstats

profiler = cProfile.Profile()
profiler.enable()

network.max_flow(
    source_path=".*/fsw.*",
    sink_path=".*/eb.*",
    mode="combine",
    shortest_path=True,
)

profiler.disable()

stats = pstats.Stats(profiler)
stats.sort_stats(pstats.SortKey.TIME).print_stats()

         6761659 function calls (6746250 primitive calls) in 1.990 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   339840    0.369    0.000    1.086    0.000 /Users/networmix/ws/NetGraph/ngraph/lib/graph.py:143(add_edge)
   339840    0.225    0.000    0.240    0.000 /Users/networmix/ws/NetGraph/ngraph-venv/lib/python3.13/site-packages/networkx/classes/coreviews.py:81(__getitem__)
   339840    0.217    0.000    0.300    0.000 /Users/networmix/ws/NetGraph/ngraph-venv/lib/python3.13/site-packages/networkx/classes/multidigraph.py:417(add_edge)
        1    0.180    0.180    0.231    0.231 /Users/networmix/ws/NetGraph/ngraph/lib/algorithms/flow_init.py:6(init_flow_graph)
        1    0.135    0.135    1.123    1.123 /Users/networmix/ws/NetGraph/ngraph/network.py:131(to_strict_multidigraph)
        1    0.132    0.132    0.154    0.154 /Users/networmix/ws/NetGraph/ngraph/lib/algorithms/calc_capacity.py:10(_init_graph_data)
   

In [15]:
explorer = NetworkExplorer.explore_network(network, scenario.components_library)

In [16]:
explorer.print_tree(skip_leaves=True, detailed=False)

- root | Nodes=6016, Links=167984, Cost=0.0, Power=10472000.0
  - dc1 | Nodes=1056, Links=32256, Cost=0.0, Power=1848000.0
    - plane1 | Nodes=132, Links=4032, Cost=0.0, Power=231000.0
      - ssw | Nodes=36, Links=4032, Cost=0.0, Power=63000.0
      - fsw | Nodes=96, Links=3456, Cost=0.0, Power=168000.0
    - plane2 | Nodes=132, Links=4032, Cost=0.0, Power=231000.0
      - ssw | Nodes=36, Links=4032, Cost=0.0, Power=63000.0
      - fsw | Nodes=96, Links=3456, Cost=0.0, Power=168000.0
    - plane3 | Nodes=132, Links=4032, Cost=0.0, Power=231000.0
      - ssw | Nodes=36, Links=4032, Cost=0.0, Power=63000.0
      - fsw | Nodes=96, Links=3456, Cost=0.0, Power=168000.0
    - plane4 | Nodes=132, Links=4032, Cost=0.0, Power=231000.0
      - ssw | Nodes=36, Links=4032, Cost=0.0, Power=63000.0
      - fsw | Nodes=96, Links=3456, Cost=0.0, Power=168000.0
    - plane5 | Nodes=132, Links=4032, Cost=0.0, Power=231000.0
      - ssw | Nodes=36, Links=4032, Cost=0.0, Power=63000.0
      - fsw | Node